In [1]:
from google.colab import drive
import os
import pandas as pd

# Mount Google Drive
drive.mount('/content/drive')
# Load DataFrame from the CSV file
csv_file_path = '/content/drive/My Drive/Projet 2CS/DATASET/dataset.csv'
df = pd.read_csv(csv_file_path)

# Now df contains the DataFrame with the data from the CSV file
print("DataFrame loaded successfully from:", csv_file_path)


Mounted at /content/drive
DataFrame loaded successfully from: /content/drive/My Drive/Projet 2CS/DATASET/dataset.csv


In [ ]:
!pip install astor
!pip install deap

In [ ]:
import traceback
import subprocess
import subprocess

def evaluate_program(erroneous_program):
    # Execute the erroneous program as a subprocess
    result = subprocess.run(['python', '-c', erroneous_program], capture_output=True, text=True)

    # Extract the number of failed tests from the errors (if any)
    failed_tests = result.stderr.count('FAIL')

    # Extract the total number of tests from the output
    start_index = result.stderr.find('Ran ')
    end_index = result.stderr.find('tests')
    total_tests = int(result.stderr[start_index + 4:end_index].strip())

    print('total_tests : ', total_tests)
    print('failed_tests : ', failed_tests)
    # Calculate the number of successful tests
    successful_tests = total_tests - failed_tests

    return total_tests, failed_tests, successful_tests


In [ ]:
#SOMETIMES THE FIRST EVALUATE DONT DOESNT WORK IDK WHY SO I USE THIS ONE INSTEAD :

'''
import subprocess
import re

def evaluate_program(erroneous_program):
    # Execute the erroneous program as a subprocess
    result = subprocess.run(['python', '-c', erroneous_program], capture_output=True, text=True)

    # Initialize counters
    total_tests = 0
    total_failed_tests = 0

    # Extract information from the stderr
    stderr_output = result.stderr

    # Use regex to find the number of tests ran
    match = re.search(r'Ran (\d+) tests', stderr_output)
    if match:
        total_tests = int(match.group(1))

    # Use regex to find the number of failures and errors
    match_failures = re.search(r'FAILED \((failures=(\d+))?(, )?(errors=(\d+))?\)', stderr_output)
    if match_failures:
        failures = match_failures.group(2)
        errors = match_failures.group(5)
        if failures:
            total_failed_tests += int(failures)
        if errors:
            total_failed_tests += int(errors)

    # Calculate the number of successful tests
    successful_tests = total_tests - total_failed_tests
    failed_tests = total_failed_tests

    print('total_tests : ', total_tests)
    print('failed_tests (including errors) : ', total_failed_tests)
    print('successful_tests : ', successful_tests)

    return total_tests, failed_tests, successful_tests

'''

"\nimport subprocess\nimport re\n\ndef evaluate_program(erroneous_program):\n    # Execute the erroneous program as a subprocess\n    result = subprocess.run(['python', '-c', erroneous_program], capture_output=True, text=True)\n\n    # Initialize counters\n    total_tests = 0\n    failed_tests = 0\n\n    # Extract information from the stderr\n    stderr_output = result.stderr\n\n    # Use regex to find the number of tests ran\n    match = re.search(r'Ran (\\d+) tests', stderr_output)\n    if match:\n        total_tests = int(match.group(1))\n\n    # Use regex to find the number of failures\n    match = re.search(r'FAILED \\(failures=(\\d+)\\)', stderr_output)\n    if match:\n        failed_tests = int(match.group(1))\n\n    # Calculate the number of successful tests\n    successful_tests = total_tests - failed_tests\n\n    print('total_tests : ', total_tests)\n    print('failed_tests : ', failed_tests)\n    print('successful_tests : ', successful_tests)\n\n    return total_tests, faile

In [ ]:
from pyparsing import Forward, Literal, Word, alphas, nums, alphanums

# Définition de la grammaire pour l'expression préfixe
expr = Forward()
identifier = Word(alphas + '_', alphanums + '_')
operand = Word(nums) | (Literal('-').suppress() + Word(nums))
op = Literal('+') | Literal('-') | Literal('*') | Literal('/')
open_paren = Literal("(").suppress()
close_paren = Literal(")").suppress()
comma = Literal(",").suppress()
expr <<= op + open_paren + expr + comma + expr + close_paren | identifier | operand

# Définition de la notation infixée avec la bonne précédence des opérateurs
def infix_action(tokens):
    if len(tokens) == 1:
        return tokens[0]
    elif tokens[0] in "+-*/":
        return f"({tokens[1]} {tokens[0]} {tokens[2]})"
    elif isinstance(tokens[1], str) and tokens[1].startswith("-"):
        return f"({tokens[1]}{tokens[0]}{tokens[2]})"
    else:
        return f"({tokens[0]}{tokens[1]})"

expr.setParseAction(infix_action)

# Fonction pour convertir une expression préfixe en infixée
def prefix_to_infix(prefix_expr):
    return expr.parseString(prefix_expr)[0]

# Exemple d'utilisation
#prefix_expr = "+(mm1234, +(+(var123   ,    *(var,   xyz)),   fgdh))"
#infix_expr = prefix_to_infix(prefix_expr)
#print("Expression infixée:", infix_expr)


In [1]:
import ast
import astor

def replace_expression(code, line_number, old_expression, new_expression):
    # Parse the code into an abstract syntax tree (AST)
    print("Old Expression:", old_expression)
    print("New Expression:", new_expression)
    tree = ast.parse(code)

    # Define a visitor to traverse the AST and perform replacements
    class ReplaceExpression(ast.NodeTransformer):
        def visit_Assign(self, node):
            # Check if the node corresponds to the specified line number
            if getattr(node, 'lineno', None) == line_number:
                print("Checking line:", astor.to_source(node).strip())
                # Try parsing the old and new expressions into AST nodes
                try:
                    old_expr_ast = ast.parse("(" + old_expression + ")", mode='eval').body
                    new_expr_ast = ast.parse("(" + new_expression + ")", mode='eval').body
                except SyntaxError as e:
                    print("SyntaxError:", e)
                    return node

                # Compare the old expression in the AST with the given old expression
                if self.compare_expr(node.value, old_expr_ast):
                    # Replace the old expression with the new expression
                    node.value = new_expr_ast

            return node

        def compare_expr(self, expr1, expr2):
            # Compare the string representations of the AST nodes
            return ast.dump(expr1) == ast.dump(expr2)

    # Instantiate the visitor and traverse the AST
    transformer = ReplaceExpression()
    transformed_tree = transformer.visit(tree)

    # Generate Python code from the modified AST
    modified_code = astor.to_source(transformed_tree)
    print('modified_code : ' , modified_code)

    return modified_code

#new_expression = ' (a - b) '
#print('erroneous_program before replaced : ', erroneous_program)
#erroneous_code = replace_expression(erroneous_program, 10 , " b - a ", str(new_expression)[2:-2].strip())
#print('erroneous_code replaced : ', erroneous_code)
# Evaluate the corrected code using the provided testing function
#num_successful_tests = evaluate_program(erroneous_code)
#print('number of successful tests : ', num_successful_tests )


ModuleNotFoundError: No module named 'astor'

In [4]:
# Assuming df is your DataFrame containing the dataset
erroneous_program = df.iloc[61]['Content']
print(erroneous_program)

lines = erroneous_program.split('\n')

for i, line in enumerate(lines, start=1):
    print(f"Line {i}: {line}")

print('lines[9] ' , lines[9])

import unittest
class AbsMinus5:
    # returns |i-j|, the absolute value of i minus j
    @staticmethod
    def absM(i, j):
        result = i + 1  # error in the assignment: result = i+1 instead of result = i

        if i < j:
            result = j - result
        else:
            result = result - j

        return result
    
# Define a test class inheriting from unittest.TestCase
class TestAddFunction(unittest.TestCase):
    def test_0(self):
        self.assertEqual(AbsMinus5.absM(1, 2), 1)  

    def test_1(self):
        self.assertEqual(AbsMinus5.absM(5, 6), 1)  

    def test_2(self):
        self.assertEqual(AbsMinus5.absM(5, 8), 3)  

    def test_3(self):
        self.assertEqual(AbsMinus5.absM(1, 6), 5)  
        
    def test_4(self):
        self.assertEqual(AbsMinus5.absM(1, 8), 7)  
    
    def test_5(self):
        self.assertEqual(AbsMinus5.absM(10, 2), 8)  
    
    def test_6(self):
        self.assertEqual(AbsMinus5.absM(10, 6), 4)  
        
    def test_7(s

In [ ]:
#old extract variables

'''
import ast
import keyword

def extract_variables_constants(erroneous_program):
    tree = ast.parse(erroneous_program)
    variables = set()
    constants = set()

    # Parcourir l'arbre syntaxique pour extraire les noms des variables et constantes
    for node in ast.walk(tree):
        if isinstance(node, ast.Name) and not isinstance(node.ctx, ast.Store):
            # Vérifier si le nom n'est pas un mot-clé réservé
            if node.id not in keyword.kwlist and node.id != "staticmethod"and node.id != "Gcd" and node.id != "gcd" and node.id != "unittest" and node.id != "self" and node.id != "__name__" and node.id != "result":
                variables.add(node.id)
        elif isinstance(node, ast.Num):
            constants.add(node.n)

    return list(variables), list(constants)

variables = extract_variables_constants(erroneous_code)

'''


'\nimport ast\nimport keyword\n\ndef extract_variables_constants(erroneous_program):\n    tree = ast.parse(erroneous_program)\n    variables = set()\n    constants = set()\n\n    # Parcourir l\'arbre syntaxique pour extraire les noms des variables et constantes\n    for node in ast.walk(tree):\n        if isinstance(node, ast.Name) and not isinstance(node.ctx, ast.Store):\n            # Vérifier si le nom n\'est pas un mot-clé réservé\n            if node.id not in keyword.kwlist and node.id != "staticmethod"and node.id != "Gcd" and node.id != "gcd" and node.id != "unittest" and node.id != "self" and node.id != "__name__" and node.id != "result":\n                variables.add(node.id)\n        elif isinstance(node, ast.Num):\n            constants.add(node.n)\n\n    return list(variables), list(constants)\n\nvariables = extract_variables_constants(erroneous_code)\n\n'

In [ ]:
#new extract variables

import ast
import keyword
import builtins

def extract_variables_constants(erroneous_program):
    tree = ast.parse(erroneous_program)
    variables = set()
    constants = set()

    # Collect function names to exclude them from variables
    function_names = {node.name for node in ast.walk(tree) if isinstance(node, ast.FunctionDef)}

    # Collect names defined at the module level (e.g., imported modules, classes)
    module_level_names = set()
    for node in ast.walk(tree):
        if isinstance(node, (ast.FunctionDef, ast.ClassDef)):
            module_level_names.add(node.name)
        elif isinstance(node, (ast.Import, ast.ImportFrom)):
            for alias in node.names:
                module_level_names.add(alias.name)

    # Add built-in function names to exclude them from variables
    builtins_names = set(dir(builtins))

    # Collect names defined within test methods
    test_method_names = set()
    for node in ast.walk(tree):
        if isinstance(node, ast.FunctionDef) and node.name.startswith("test_"):
            # Exclude variables defined within test methods
            for child_node in ast.walk(node):
                if isinstance(child_node, ast.Name) and not isinstance(child_node.ctx, ast.Store):
                    test_method_names.add(child_node.id)


    for node in ast.walk(tree):
        if isinstance(node, ast.Name) and not isinstance(node.ctx, ast.Store):
            # Check if the name is not a keyword, not a function name, not a built-in,
            # not a module-level name, and not a name used in test cases
            if node.id not in keyword.kwlist and node.id not in function_names \
               and node.id not in module_level_names and node.id not in builtins_names \
               and node.id not in test_method_names:
                variables.add(node.id)
        elif isinstance(node, ast.Constant) and node.value != '__main__' :  # For Python 3.8+
            if isinstance(node.value, (int, float, str)):
                constants.add(node.value)
        elif isinstance(node, ast.Num):  # For compatibility with Python 3.7 and below
            constants.add(node.n)

    return list(variables), list(constants)


variables = extract_variables_constants(erroneous_program)
print(variables)


(['b', 'a'], [0, 5, 10, 15, 20])


In [ ]:
import operator
import random
import numpy as np
import math
import csv  # Import the CSV module
from deap import algorithms, base, creator, tools, gp
import operator
import random
import numpy as np
import math
import csv
import time
import deap
from deap import algorithms, base, creator, tools, gp

# Define global variables
max_generations = 0
population_size = 100
total_tests = 25
output_directory = "/content/drive/My Drive/Projet 2CS/Statistics/GCD/"
logbook =''


# Define the primitive set for the symbolic regression problem
pset = gp.PrimitiveSet("MAIN", arity=len(variables))  # Set arity equal to the number of variables
pset.addPrimitive(operator.add, arity=2)
pset.addPrimitive(operator.sub, arity=2)
pset.addPrimitive(operator.mul, arity=2)
pset.addEphemeralConstant("rand101", lambda: random.randint(-10, 10))

creator.create("FitnessMin", base.Fitness, weights=(-1.0,))
creator.create("Individual", gp.PrimitiveTree, fitness=creator.FitnessMin)

toolbox = base.Toolbox()
toolbox.register("expr", gp.genHalfAndHalf, pset=pset, min_=1, max_=2)
toolbox.register("individual", tools.initIterate, creator.Individual, toolbox.expr)
toolbox.register("population", tools.initRepeat, list, toolbox.individual)
toolbox.register("compile", gp.compile, pset=pset)

def translate_expr(individual, variables):
    expr = str(individual)
    expr = expr.replace("add", "+")
    expr = expr.replace("sub", "-")
    expr = expr.replace("mul", "*")
    expr = expr.replace("rand101", str(random.randint(-10, 10)))
    for i, var in enumerate(variables):
        arg_name = "ARG{}".format(i)
        expr = expr.replace(arg_name, var)
    return expr



import signal
class TimeoutException(Exception):
    pass

def timeout_handler(signum, frame):
    raise TimeoutException


def evalSymbReg(individual, variables):
    new_expression = translate_expr(individual, variables)
    new_expression = expr.parseString(str(new_expression))
    erroneous_code = replace_expression(erroneous_program, 10 , " b - a ", str(new_expression)[2:-2].strip())

    # Set up the timeout handler
    signal.signal(signal.SIGALRM, timeout_handler)
    timeout_seconds = 1  # Adjust the timeout as needed

    try:
        # Start the timer. This line raises an alarm signal after the specified number of seconds
        signal.alarm(timeout_seconds)
        # Your evaluation logic
        total_tests, failed_tests, successful_tests = evaluate_program(erroneous_code)
        # Cancel the alarm if the function returns before the timeout
        signal.alarm(0)

        num_failed_tests = failed_tests
        return num_failed_tests,

    except TimeoutException:
        print("Evaluation timed out!")
        total_tests = 25
        return total_tests,

    finally:
        # Make sure to cancel the alarm in case of unexpected errors
        signal.alarm(0)


def main():
    start_time = time.time()
    random.seed(42)
    variables, _ = extract_variables_constants(erroneous_program)

    if len(variables) < 1:
        print("Erreur: Nombre incorrect de variables extraites.")
        return

    variables_list = list(variables)
    print('variables_list : ' , variables_list)
    pop = toolbox.population(n=100)
    hof = tools.HallOfFame(1)
    stats = tools.Statistics(lambda ind: ind.fitness.values)
    stats.register("avg", np.mean)
    stats.register("std", np.std)
    stats.register("min", np.min)
    stats.register("max", np.max)

    toolbox.register("evaluate", evalSymbReg , variables=variables_list)

    toolbox.register("select", tools.selTournament, tournsize=3)
    toolbox.register("mate", gp.cxOnePoint)
    toolbox.register("expr_mut", gp.genFull, min_=0, max_=2)
    toolbox.register("mutate", gp.mutUniform, expr=toolbox.expr_mut, pset=pset)

    algorithm_details = []  # List to store algorithm details

    # Define hall of fame to track best individuals
    hof = tools.HallOfFame(maxsize=1)

    # Evolutionary algorithm
    pop = toolbox.population(n=population_size)
    logbook = algorithms.eaSimple(pop, toolbox, 0.5, 0.1, max_generations, stats=stats, halloffame=hof, verbose=True)

    execution_time = time.time() - start_time

    metrics_file = output_directory + "gcd_evolution_metrics_tstdina.csv"
    with open(metrics_file, mode='w', newline='') as file:
        writer = csv.writer(file)
        writer.writerow(["Generation", "Individus" , "Average Fitness", "Std Fitness", "Min Fitness", "Max Fitness", "Execution Time", "Best Individual"])
        best_individual = hof[0]
        best_expr_str = translate_expr(best_individual, variables_list)
        best_expr_str2 = expr.parseString(str(best_expr_str))
        best_expr_str2 = str(best_expr_str2)[2:-2].strip()
        for gen, record in enumerate(logbook[1:]):
            writer.writerow([record[0]['gen'], record[0]['nevals'], record[0]['avg'], record[0]['std'], record[0]['min'], record[0]['max'], execution_time, best_expr_str2 ])


    print("Metrics and best individuals saved successfully.")


if __name__ == "__main__":
    main()


/usr/local/lib/python3.10/dist-packages/deap/gp.py:254: RuntimeWarning: Ephemeral rand101 function cannot be pickled because its generating function is a lambda function. Use functools.partial instead.
  warnings.warn("Ephemeral {name} function cannot be "
/usr/local/lib/python3.10/dist-packages/deap/creator.py:185: RuntimeWarning: A class named 'FitnessMin' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  warnings.warn("A class named '{0}' has already been created and it "
/usr/local/lib/python3.10/dist-packages/deap/creator.py:185: RuntimeWarning: A class named 'Individual' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  warnings.warn("A class named '{0}' has already been created and it "


Streaming output truncated to the last 5000 lines.
        self.assertEqual(5, result)

    def test_6(self):
        result = Gcd.gcd(10, 15)
        self.assertEqual(5, result)

    def test_7(self):
        result = Gcd.gcd(15, 20)
        self.assertEqual(5, result)

    def test_8(self):
        result = Gcd.gcd(0, 10)
        self.assertEqual(10, result)

    def test_9(self):
        result = Gcd.gcd(5, 15)
        self.assertEqual(5, result)

    def test_10(self):
        result = Gcd.gcd(10, 20)
        self.assertEqual(10, result)

    def test_11(self):
        result = Gcd.gcd(20, 0)
        self.assertEqual(20, result)

    def test_12(self):
        result = Gcd.gcd(0, 15)
        self.assertEqual(15, result)

    def test_13(self):
        result = Gcd.gcd(5, 20)
        self.assertEqual(5, result)

    def test_14(self):
        result = Gcd.gcd(15, 0)
        self.assertEqual(15, result)

    def test_15(self):
        result = Gcd.gcd(20, 5)
        self.assertEqual(